# 01_load_adam_xpt_files
**Purpose: Quick demo to load SDTM Pilot 01 (Updated) data & make some simple plots**

**Goals**
1. Load a few SDTM XPT files from `../data/raw/`
2. Explain what each file contains (briefly)
3. Save tiny previews and simple plots to `../figures/` for GitHub-friendly display

**Data Source**
- SDTM examples (repo root): https://github.com/phuse-org/phuse-scripts/tree/master/data/sdtm  
- Original pilot data: https://github.com/phuse-org/phuse-scripts/tree/master/data/sdtm/cdiscpilot01  
- Updated pilot data: https://github.com/phuse-org/phuse-scripts/tree/master/data/sdtm/updated_cdiscpilot  
**Using:** the updated version.

**Notes**
- Outputs are cleared on commit (nbstripout).
- Every plot/table preview is saved to disk and then embedded via Markdown.


## Datasets in this demo (high-level)

| File | What it is (typical content) |
|---|---|
| `define.xml` | Define-XML metadata describing datasets, variables, codelists, origins, and value-level metadata. |
| `define2-0-0.xsl` | Stylesheet to render `define.xml` nicely in a browser. |
| `Readme.md` | Package notes about the data. |
| `dm.xpt` | Demographics: one record per subject (USUBJID); sex, age, race, site, arm assignment, etc. |
| `sv.xpt` | Subject Visits: per-subject visit information and timing. |
| `sc.xpt` | Subject Characteristics: baseline characteristics not captured in DM. |
| `se.xpt` | Subject Elements: protocol elements each subject passed through (e.g., Screening, Treatment). |
| `ta.xpt` | Trial Arms: definition of treatment arms. |
| `te.xpt` | Trial Elements: protocol elements (e.g., Screening, Treatment, Follow-up). |
| `ti.xpt` | Trial Inclusion/Exclusion: text of criteria from the protocol. |
| `ts.xpt` | Trial Summary: study-level parameters (e.g., start/end, durations). |
| `tv.xpt` | Trial Visits: planned visit schedule. |
| `ae.xpt` | Adverse Events: event term, timing, severity (AESEV), seriousness (AESER), outcome, causality. |
| `cm.xpt` | Concomitant Medications: medications taken during the study. |
| `ds.xpt` | Disposition: completion, discontinuation, screen failures, and reasons. |
| `ex.xpt` | Exposure: actual dosing/exposure records. |
| `mh.xpt` | Medical History: baseline conditions per subject. |
| `vs.xpt` | Vital Signs: VSTEST/VSTESTCD, numeric results (VSSTRESN), units (VSSTRESU), timing (VSSTDTC/VSSTDY). |
| `lbch.xpt` | Labs – Clinical Chemistry (split LB domain in this pilot). |
| `lbhe.xpt` | Labs – Hematology (split LB domain). |
| `lbur.xpt` | Labs – Urinalysis (split LB domain). |
| `qsco.xpt` | Questionnaire subdomain (instrument-specific). |
| `qsda.xpt` | Questionnaire subdomain (instrument-specific). |
| `qsgi.xpt` | Questionnaire subdomain (instrument-specific). |
| `qshi.xpt` | Questionnaire subdomain (instrument-specific). |
| `qsmm.xpt` | Questionnaire subdomain (instrument-specific). |
| `qsni.xpt` | Questionnaire subdomain (instrument-specific). |
| `relrec.xpt` | Related Records: links between records across domains (RELID groups). |
| `suppae.xpt` | Supplemental qualifiers for AE (extra variables not in core AE). |
| `suppdm.xpt` | Supplemental qualifiers for DM. |
| `suppds.xpt` | Supplemental qualifiers for DS. |
| `supplbch.xpt` | Supplemental qualifiers for LBCH. |
| `supplbhe.xpt` | Supplemental qualifiers for LBHE. |
| `supplbur.xpt` | Supplemental qualifiers for LBUR. |

*Note:* This pilot uses split **LB** domains and instrument-specific **QS** subdomains. In many submissions you’ll see single `LB` and `QS` domains with value-level metadata instead.


# Imports

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import pyreadstat
import dataframe_image as dfi
import xml.etree.ElementTree as ET



# Functions

In [ ]:
DATA_DIR = Path("../data/raw").resolve()
FIG_DIR  = Path("../figures").resolve()

def savefig(path, dpi=110):
    plt.savefig(path, dpi=dpi, bbox_inches="tight", pil_kwargs={"quality": 80, "optimize": True})
    plt.close()

# def save_table_image(df, out_path, caption=None, max_rows=8, max_cols=12):
#     if df is None or df.empty:
#         return
#     sample = df.iloc[:max_rows, :max_cols].copy()
#     dfi.export(sample.style.set_caption(caption or ""), out_path)

def read_xpt(path: Path) -> pd.DataFrame:
    df, meta = pyreadstat.read_xport(str(path))  # str() for libs that expect strings
    df.attrs["meta"] = meta
    return df




# Load & Peek at Data

In [ ]:
# Load & Peek at Data — SDTM (simple random sampling)
# from IPython.display import display

SEED = 17  # single global seed for reproducible random rows

files_to_load = {
    "DM":   DATA_DIR / "dm.xpt",
    "AE":   DATA_DIR / "ae.xpt",
    "VS":   DATA_DIR / "vs.xpt",
    "LBCH": DATA_DIR / "lbch.xpt",
    "LBHE": DATA_DIR / "lbhe.xpt",
    "LBUR": DATA_DIR / "lbur.xpt",
}

domain_descriptions = {
    "DM":   "Demographics",
    "AE":   "Adverse Events",
    "VS":   "Vital Signs",
    "LBCH": "Labs – Clinical Chemistry",
    "LBHE": "Labs – Hematology",
    "LBUR": "Labs – Urinalysis",
}

preferred_cols = {
    "DM":   ["STUDYID","USUBJID","SITEID","ARM","ARMCD","SEX","AGE","RACE","COUNTRY"],
    "AE":   ["STUDYID","USUBJID","AETERM","AEDECOD","AESEV","AESER","AEREL","AESTDTC","AEENDTC","AEOUT"],
    "VS":   ["STUDYID","USUBJID","VSTEST","VSTESTCD","VSORRES","VSSTRESN","VSSTRESU","VSSTDTC","VSSTDY","VISIT"],
    "LBCH": ["STUDYID","USUBJID","LBTEST","LBTESTCD","LBORRES","LBSTRESN","LBSTRESU","LBNRIND","LBDTC","VISIT"],
    "LBHE": ["STUDYID","USUBJID","LBTEST","LBTESTCD","LBORRES","LBSTRESN","LBSTRESU","LBNRIND","LBDTC","VISIT"],
    "LBUR": ["STUDYID","USUBJID","LBTEST","LBTESTCD","LBORRES","LBSTRESN","LBSTRESU","LBNRIND","LBDTC","VISIT"],
}

def pick_cols(df, wanted):
    cols = [c for c in wanted if c in df.columns]
    return cols if cols else list(df.columns)[:10]

save_index = -1
for name, path in files_to_load.items():
    if not path.exists():
        print(f"WARNING: {path.name} not found in {DATA_DIR}")
        continue

    desc = domain_descriptions.get(name, "")
    header = f"{name} — {desc}" if desc else name
    print("=" * len(header))
    print(header)
    print("=" * len(header))
    print(f"File: {path.name}\n")

    df = read_xpt(path)
    cols = pick_cols(df, preferred_cols.get(name, []))

    print(f"Shape: {df.shape}")
    n = min(5, len(df))
    display(df.loc[:, cols].sample(n=n, random_state=SEED))
    print()  # spacer

    # Same 5 random rows for the JPEG preview
    preview = df.loc[:, cols].sample(n=n, random_state=SEED).copy()
    styled = (
        preview.style
        .set_caption(f"{name} Sample Rows")
        .hide(axis="index")
        .set_properties(**{"font-size": "9pt", "white-space": "nowrap"})
    )

    save_index += 1
    out_path = FIG_DIR / f"{save_index:02d}_df_preview_{name.lower()}.jpg"
    dfi.export(styled, out_path, table_conversion="matplotlib", dpi=150)


### Preview of DM Demographics
<img src="../figures/00_df_preview_dm.jpg?v=2" alt="DM sample rows" width="1000">

### Preview of AE Adverse Events
<img src="../figures/01_df_preview_ae.jpg?v=2" alt="AE sample rows" width="1000">

### Preview of VS Vital Signs
<img src="../figures/02_df_preview_vs.jpg?v=2" alt="VS sample rows" width="1000">

### Preview of LBCH Clinical Chemistry Labs
<img src="../figures/03_df_preview_lbch.jpg?v=2" alt="LBCH sample rows" width="1000">

### Preview of LBHE Hematology Labs
<img src="../figures/04_df_preview_lbhe.jpg?v=2" alt="LBHE sample rows" width="1000">

### Preview of LBUR Urinalysis Labs
<img src="../figures/05_df_preview_lbur.jpg?v=2" alt="LBUR sample rows" width="1000">


# How to quickly look-up column meanings — SDTM


In [ ]:
define_path = DATA_DIR / "define.xml"   # ../data/raw/define.xml
root = ET.parse(str(define_path)).getroot()

def sdtm_var_label(dataset: str, var: str):
    """
    Return the Define-XML label for `var` in SDTM dataset `dataset` (e.g., 'VS', 'DM').
    """
    for ig in root.findall(f".//{{*}}ItemGroupDef[@Name='{dataset}']"):
        for itemref in ig.findall(".//{*}ItemRef"):
            oid = itemref.get("ItemOID")
            item = root.find(f".//{{*}}ItemDef[@OID='{oid}']")
            if item is not None and item.get("Name") == var:
                txt = item.find(".//{*}Description/{*}TranslatedText")
                return txt.text.strip() if txt is not None else None
    return None

print("Examples")
print("DM data file, column AGE:       ", sdtm_var_label("DM", "AGE"))
print("DM data file, column SEX:       ", sdtm_var_label("DM", "SEX"))
print("DS data file, column DSDECOD:   ", sdtm_var_label("DS", "DSDECOD"))
print("EX data file, column EXSTDTC:   ", sdtm_var_label("EX", "EXSTDTC"))
print("VS data file, column VSDTC:     ", sdtm_var_label("VS", "VSDTC"))
print("VS data file, column VSSTRESN:  ", sdtm_var_label("VS", "VSSTRESN"))
print("VS data file, column VSSTRESU:  ", sdtm_var_label("VS", "VSSTRESU"))
print("LBCH data file, column LBSTRESN:", sdtm_var_label("LBCH", "LBSTRESN"))


** Quick column meaning lookups (from Define-XML) **

- **DM** data file — `AGE`: **Age**
- **DM** data file — `SEX`: **Sex**
- **DS** data file — `DSDECOD`: **Standardized Disposition Term**
- **EX** data file — `EXSTDTC`: **Start Date/Time of Treatment**
- **VS** data file — `VSDTC`: **Date/Time of Measurements**
- **VS** data file — `VSSTRESN`: **Numeric Result/Finding in Standard Units**
- **VS** data file — `VSSTRESU`: **Standard Units**
- **LBCH** data file — `LBSTRESN`: **Numeric Result/Finding in Standard Units**

# Simple Summaries

In [ ]:
"""
Patients per Treatment Arm per Site — SDTM (include Screen Failure)
- Screen Failure Info: this is an example of information in STDM that isn't in ADaM.
"""
dm = read_xpt(DATA_DIR / "dm.xpt").copy()

# Make SITEID numeric if possible (for cleaner x-axis)
try:
    dm["SITEID"] = dm["SITEID"].astype(int)
except Exception:
    pass

# Order arms by overall count (keeps all levels, incl. Screen Failure)
arm_order = dm["ARM"].value_counts().index.tolist()

site_counts = (
    dm.groupby(["SITEID", "ARM"])["USUBJID"]
      .nunique()
      .reset_index(name="n_patients")
)

pivot = (
    site_counts.pivot(index="SITEID", columns="ARM", values="n_patients")
              .reindex(columns=arm_order)  # keep consistent col order
              .fillna(0)
              .sort_index()
)

ax = pivot.plot(kind="bar", figsize=(10, 5))
plt.ylabel("Number of Patients")
plt.title("Patients per Treatment Arm by Site\n(Screen Failure Included)")
plt.tight_layout()

# Save image
save_index += 1
out_path = FIG_DIR / f"{save_index:02d}_patients_per_site.jpg"
savefig(out_path, dpi=100)


In [ ]:
dm = read_xpt(DATA_DIR / "dm.xpt")

treatment_cols = [c for c in ["ARMCD", "ARM", "ACTARMCD", "ACTARM"] if c in dm.columns]

# Build side-by-side Value/Count pairs per column (keep NaNs as blank)
frames = []
for col in treatment_cols:
    vc = dm[col].value_counts(dropna=False)
    dfc = vc.reset_index()
    dfc.columns = [f"{col} Value", f"{col} Count"]
    dfc[f"{col} Value"] = dfc[f"{col} Value"].astype(object).where(dfc[f"{col} Value"].notna(), "")
    dfc[f"{col} Count"] = dfc[f"{col} Count"].astype("Int64")
    frames.append(dfc)

max_len = max((len(f) for f in frames), default=0)
for i, col in enumerate(treatment_cols):
    dfc = frames[i].reindex(range(max_len))
    val_col, cnt_col = f"{col} Value", f"{col} Count"
    dfc[val_col] = dfc[val_col].fillna("")
    dfc[cnt_col] = dfc[cnt_col].astype("Int64")
    dfc[cnt_col] = dfc[cnt_col].astype(object).where(dfc[cnt_col].notna(), "")
    frames[i] = dfc

arm_value_counts = pd.concat(frames, axis=1)

# ---- Multiline header: field name on line 1, definition on line 2 ----
col_defs = {
    "ARM": "Planned arm",
    "ARMCD": "Planned arm code",
    "ACTARM": "Actual arm",
    "ACTARMCD": "Actual arm code",
}

pairs = []
for base in ["ARMCD", "ARM", "ACTARMCD", "ACTARM"]:
    val_col = f"{base} Value"
    cnt_col = f"{base} Count"
    if val_col in arm_value_counts.columns:
        hdr = f"{base}\n{col_defs.get(base, '')}"  # <-- newline instead of dash
        pairs.append((hdr, "Value"))
        pairs.append((hdr, "Count"))

mi = arm_value_counts.copy()
mi.columns = pd.MultiIndex.from_tuples(pairs)

styled = (
    mi.style
      .set_caption("Treatment / Arm Value Counts (SDTM DM)")
      .hide(axis="index")
      # Optional: make header wrapping/alignment a bit nicer (has effect mainly in HTML export)
      .set_table_styles([
          {"selector": "th.col_heading", "props": [("text-align", "left")]},
          {"selector": "th.col_heading.level0", "props": [("white-space", "pre-wrap")]},
          {"selector": "th.col_heading.level1", "props": [("white-space", "nowrap")]},
      ])
)


## Save Table
save_index += 1
out_path = FIG_DIR / f"{save_index:02d}_stdm_treatment_var_defs_and_counts.jpg"
dfi.export(styled, out_path, table_conversion="matplotlib", dpi=150)



### STDM Data has information that's not in ADaM
** Patients per Site **
<img src="../figures/06_patients_per_site.jpg?v=2" alt="Patients per Site" width="1000">

** Treatment Variable Information **
<img src="../figures/07_stdm_treatment_var_defs_and_counts.jpg?v=2" alt="Treatment Varaibles" width="1000">


# STDM Vital Signs
- Using the `vs.xpt` data set you can look at vital signs within & across patients.
- The trends you see in here are common in other vital sign data sets, for example...
    - Compared to the population range, individual patients occupy a much smaller range of the values
    - So values that are entirely rare to the population might be the norm for an individual patient, and vice versa.
    - Repeated measurements are not a silver bullet: there's huge variation in vitals, even when taken within the same visit. --> so imagine all the variability that occurs between site visits.
- Given these similarities to a couple dozen of the other vital sign data sets I've seen, I'd say it's not a bad example data set.

In [ ]:
# SDTM Vital Signs — list available tests with counts (include NaNs)

COUNT_FIELD = "VSTEST"   # human-readable names
INCLUDE_NA  = True       # include NaNs in value_counts

# Load VS
df_vs = read_xpt(DATA_DIR / "vs.xpt")

if COUNT_FIELD not in df_vs.columns:
    raise KeyError(f"{COUNT_FIELD} not found in VS columns: {list(df_vs.columns)}")

# Build counts (include NaNs) and sort by count desc, then name asc
vc = (
    df_vs[COUNT_FIELD]
      .value_counts(dropna=not INCLUDE_NA)  # INCLUDE_NA=True -> dropna=False
      .rename("n")
      .reset_index()
      .rename(columns={"index": COUNT_FIELD})
      .sort_values(["n", COUNT_FIELD], ascending=[False, True])
)

# Show NaNs as blank in the Value column
vc[COUNT_FIELD] = vc[COUNT_FIELD].astype(object).where(vc[COUNT_FIELD].notna(), "")

# Style
styled = (
    vc.style
      .set_caption("Available Vital Sign Tests")
      .hide(axis="index")
      .format({"n": "{:,}"})
)

# Save image for Markdown embedding
try:
    save_index
except NameError:
    save_index = -1
save_index += 1
fig_str = f"{save_index:02d}"
out_path = FIG_DIR / f"{fig_str}_vitalsign_params_available_vs.jpg"

dfi.export(
    styled,
    out_path,
    table_conversion="matplotlib",
    dpi=150
)

### Available Vital Sign Tests (VSTEST)
<img src="../figures/08_vitalsign_params_available_vs.jpg?v=1" alt="Available VS tests with counts (including missing)" width="300">


In [ ]:
"""
Get Vital Sign Values & Statistics: Within & Across Patients (SDTM)
- Test: VSTEST == "Pulse Rate"
- Value: VSSTRESN (numeric standardized result)
- Include all subjects (screen failures highlighted later)
"""

# Load VS and DM
vs = read_xpt(DATA_DIR / "vs.xpt")
dm = read_xpt(DATA_DIR / "dm.xpt")

# Filter to Pulse Rate and coerce numeric values
vs_pulse = vs.loc[vs["VSTEST"] == "Pulse Rate"].copy()
vs_pulse["VSSTRESN"] = pd.to_numeric(vs_pulse["VSSTRESN"], errors="coerce")

# Subject-wise quartiles (drop subjects with all-NaN)
pulse_quants = (
    vs_pulse.groupby("USUBJID")["VSSTRESN"]
            .quantile([0.25, 0.5, 0.75])
            .unstack()
            .reset_index()
            .rename(columns={0.25: "p25", 0.5: "p50", 0.75: "p75"})
            .dropna(subset=["p50"])
            .sort_values("p50")
            .reset_index(drop=True)
)

# Map subject -> sorted index (for x-axis)
index_map = dict(zip(pulse_quants["USUBJID"], pulse_quants.index))

# Build plotting vectors for all observations (drop NaNs in y)
valid = vs_pulse["VSSTRESN"].notna()
x_vals = vs_pulse.loc[valid, "USUBJID"].map(index_map)
y_vals = vs_pulse.loc[valid, "VSSTRESN"]

# Mark screen failures for orange overlay
vs_pulse = vs_pulse.merge(dm[["USUBJID", "ARM"]], on="USUBJID", how="left")
sf_mask = valid & (vs_pulse["ARM"] == "Screen Failure")
x_vals_sf = vs_pulse.loc[sf_mask, "USUBJID"].map(index_map)
y_vals_sf = vs_pulse.loc[sf_mask, "VSSTRESN"]


In [ ]:
# Toggle: True = bars go left→right; False = right→left
bars_right = True



bins = np.arange(40, 121, 2.5)

fig = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(1, 2, width_ratios=[4, 1], wspace=0.05)

# --- Main plot (left) ---
ax_main = plt.subplot(gs[0])
# All observations (cyan)
ax_main.plot(x_vals, y_vals, ".c", alpha=0.5, label="Heart Rate Obs")
# Screen failures overlay (orange)
ax_main.plot(x_vals_sf, y_vals_sf, ".", color="orange", alpha=0.8, label="Screen Failure Obs")

# Subject-specific IQR whiskers + medians
ax_main.vlines(x=pulse_quants.index, ymin=pulse_quants.p25, ymax=pulse_quants.p75,
               colors="b", label="Subject Q25–Q75")
ax_main.plot(pulse_quants.index, pulse_quants.p50, "-r", label="Subject Median")

ax_main.grid(True)
ax_main.set_xlabel("Subject Index (Sorted by Median)", fontsize=16)
ax_main.set_ylabel("Pulse Rate (Beats/Min)", fontsize=16)
ax_main.set_title("SDTM: Heart Rate (Pulse) Distributions: Within vs Across Subjects", fontsize=16)
ax_main.legend(fontsize=14)

# --- Histogram (right) ---
ax_hist = plt.subplot(gs[1], sharey=ax_main)
ax_hist.hist(y_vals, bins=bins, orientation="horizontal", alpha=0.7)

if bars_right:
    ax_hist.yaxis.tick_right()
    ax_hist.yaxis.set_label_position("right")
else:
    ax_hist.invert_xaxis()
    ax_hist.yaxis.tick_left()
    ax_hist.yaxis.set_label_position("left")

ax_hist.set_xlabel("Count")
ax_hist.set_ylabel("Pulse Rate (Beats/Min)", fontsize=14, labelpad=10)
ax_hist.tick_params(axis="y", labelsize=12)
ax_hist.set_title("Overall Distribution", fontsize=14)
ax_hist.grid(True)

# --- Save ---
save_index += 1
fig_str = f"{save_index:02d}"
out_path = FIG_DIR / f"{fig_str}_heart_rate_quantiles_plus_hist.jpg"
savefig(out_path, dpi=100)


Quick Interpretation:
- Compared to the population range, individual patients occupy a much smaller range of the values
- So values that are entirely rare to the population might be the norm for an individual patient, and vice versa.
- STDM-Specific: Note that screening-failure subjects didn't have any recorded heart rate values.

<img src="../figures/09_heart_rate_quantiles_plus_hist.jpg?v=1" alt="Heart Rate Quantiles and Hist" width="1000">


In [ ]:
"""
Subject vital-sign variation over time (SDTM)
Figure A: x-axis = VSDTC (datetime)
Figure B: x-axis = VSDY (study day)
Tests: Pulse Rate, Temperature, Systolic BP, Diastolic BP
Values: VSSTRESN; Units shown from VSSTRESU (mode per test)
"""

# import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
# import pandas as pd

vs = read_xpt(DATA_DIR / "vs.xpt")

subj_id = "01-701-1015"
tests = ["Pulse Rate", "Temperature", "Systolic Blood Pressure", "Diastolic Blood Pressure"]

# Filter to subject and needed columns once
need_cols = ["USUBJID","VSTEST","VSDTC","VSDY","VSSTRESN","VSSTRESU","VISITNUM"]
vs_subj = vs.loc[vs["USUBJID"] == subj_id, [c for c in need_cols if c in vs.columns]].copy()

# Types
vs_subj["VSSTRESN"] = pd.to_numeric(vs_subj.get("VSSTRESN"), errors="coerce")
if "VSDY" in vs_subj.columns:
    vs_subj["VSDY"] = pd.to_numeric(vs_subj["VSDY"], errors="coerce")
if "VSDTC" in vs_subj.columns:
    vs_subj["VSDTC_dt"] = pd.to_datetime(vs_subj["VSDTC"], errors="coerce")
if "VISITNUM" in vs_subj.columns:
    vs_subj["VISITNUM"] = pd.to_numeric(vs_subj.get("VISITNUM"), errors="coerce").astype("Float64")

# Pre-compute modal unit per test (from full VS for robustness)
def unit_for_test(df, test):
    s = df.loc[df["VSTEST"] == test, "VSSTRESU"].dropna()
    return s.mode().iloc[0] if not s.empty else ""

units = {t: unit_for_test(vs, t) for t in tests}

# --- Generate both figures: datetime and study day ---
x_axes = [
    ("VSDTC_dt", "Date/Time (VSDTC)", "vsdtc"),
    # ("VSDY",     "Study Day (VSDY)",  "vsdy"),
    ("VISITNUM", "Visit Number (VISITNUM)", "visitnum"),
]

for xcol, xlabel, suffix in x_axes:
    if xcol not in vs_subj.columns:
        print(f"Skipping figure for {xcol}: column not found.")
        continue

    fig = plt.figure(figsize=(20, 10))
    plt.suptitle(f"SDTM: Subject Vital Signs — {subj_id}", fontsize=24)
    gs = gridspec.GridSpec(2, 2, wspace=0.25, hspace=0.25)

    for i, test in enumerate(tests):
        df_t = vs_subj.loc[
            (vs_subj["VSTEST"] == test)
            & vs_subj[xcol].notna()
            & vs_subj["VSSTRESN"].notna()
        ]
        ax = fig.add_subplot(gs[i // 2, i % 2])
        ax.scatter(df_t[xcol], df_t["VSSTRESN"], marker=".", s=50)
        ax.set_ylabel(f"{test} ({units.get(test,'')})", fontsize=18)
        ax.set_xlabel(xlabel, fontsize=18)
        ax.tick_params(axis="x", labelsize=14, rotation=20 if xcol == "VSDTC_dt" else 0)
        ax.tick_params(axis="y", labelsize=14)
        ax.grid(True)

    # Save 
    save_index += 1
    fig_str = f"{save_index:02d}"
    out_path = FIG_DIR / f"{fig_str}_subject_vitals_timeseries_{suffix}.jpg"
    savefig(out_path, dpi=100)


Quick Interpretation:
- To state the obvious, viset visits aren't evenly distributed across time.
- Repeated measurements are not a silver bullet: there's huge variation in vitals, even when taken within the same visit. 
- Imagine all the variability that occurs between site visits.


<img src="../figures/11_subject_vitals_timeseries_visitnum.jpg?v=1" alt="Heart Rate Quantiles and Hist" width="1000">

<img src="../figures/10_subject_vitals_timeseries_vsdtc.jpg?v=1" alt="Heart Rate Quantiles and Hist" width="1000">


In [ ]:
"""
SDTM Lab Values — Chemistry (LBCH)
Show Available Lab Tests with Value Counts
- Domain: LBCH only
- Count field: LBTEST (human-readable names)
- Include missing values
"""

lbch = read_xpt(DATA_DIR / "lbch.xpt")

COUNT_FIELD = "LBTEST"

preview = (
    lbch[COUNT_FIELD]
      .value_counts(dropna=False)   # INCLUDE_NA=True -> dropna=False
      .rename("n")
      .reset_index()
      .rename(columns={"index": COUNT_FIELD})
      .sort_values(["n", COUNT_FIELD], ascending=[False, True])
)

# Show NaNs as blank for display
preview[COUNT_FIELD] = preview[COUNT_FIELD].astype(object).where(preview[COUNT_FIELD].notna(), "")

styled = (
    preview.style
      .set_caption("Available Lab Tests (LBCH · LBTEST)")
      .hide(axis="index")
      .format({"n": "{:,}"})
)

# Save image for Markdown embedding
save_index += 1
fig_str = f"{save_index:02d}"
out_path = FIG_DIR / f"{fig_str}_lab_params_available_lbch.jpg"

dfi.export(
    styled,
    out_path,
    table_conversion="matplotlib",
    dpi=150
)



<img src="../figures/12_lab_params_available_lbch.jpg?v=2" alt="Available Lab Params" width="300">


In [ ]:
"""
Bilirubin Values & Statistics — Within & Across Patients (SDTM LBCH)
Prep subject-wise quantiles and (x_vals, y_vals) vectors for plotting.
- Filter: LBTEST == "Bilirubin"
- Value:  LBSTRESN (assumed standardized)
- Unit:   modal LBSTRESU shown for labeling
"""

lbch = read_xpt(DATA_DIR / "lbch.xpt")

# Filter to Bilirubin rows and coerce numeric
bili = lbch.loc[lbch["LBTEST"] == "Bilirubin"].copy()
bili["LBSTRESN"] = pd.to_numeric(bili["LBSTRESN"], errors="coerce")

# Subject-wise quartiles (drop subjects with all-NaN)
bili_quants = (
    bili.groupby("USUBJID")["LBSTRESN"]
        .quantile([0.25, 0.5, 0.75])
        .unstack()
        .reset_index()
        .rename(columns={0.25: "p25", 0.5: "p50", 0.75: "p75"})
        .dropna(subset=["p50"])
        .sort_values("p50")
        .reset_index(drop=True)
)

# Map subject -> position on x-axis (sorted by median)
index_map = dict(zip(bili_quants["USUBJID"], bili_quants.index))

# Build plotting vectors for all observations belonging to subjects in bili_quants
mask = bili["USUBJID"].isin(index_map)
x_vals = bili.loc[mask, "USUBJID"].map(index_map)
y_vals = bili.loc[mask, "LBSTRESN"]

# Modal unit for labeling your plot (e.g., "umol/L")
unit_series = bili["LBSTRESU"].dropna()
bili_unit = unit_series.mode().iloc[0] if not unit_series.empty else ""
print(f"Prepared {len(y_vals)} observations across {bili_quants.shape[0]} subjects. Modal unit: {bili_unit}")


In [ ]:
# Toggle: True = bars go left→right (clockwise look); False = right→left (counterclockwise look)
bars_right = True

# Clean vectors (align indices; drop NaNs for plotting)
mask_valid = x_vals.notna() & y_vals.notna()
x_plot = x_vals[mask_valid]
y_plot = y_vals[mask_valid]

# Dynamic bins: keep ADaM-style baseline (0→42 by 2), extend if data exceed
ymax = float(np.nanmax(y_plot)) if len(y_plot) else 0.0
upper = max(42.0, np.ceil(ymax / 2.0) * 2.0 + 2.0)
bins = np.arange(0.0, upper + 1e-9, 2.0)

fig = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(1, 2, width_ratios=[4, 1], wspace=0.05)

# --- Main plot (left) ---
ax_main = plt.subplot(gs[0])
ax_main.plot(x_plot, y_plot, ".c", alpha=0.5, label="Bilirubin Obs")
ax_main.vlines(
    x=bili_quants.index, ymin=bili_quants.p25, ymax=bili_quants.p75,
    colors="b", label="Patient-Specific Q25–Q75"
)
ax_main.plot(bili_quants.index, bili_quants.p50, "-r", label="Patient-Specific Median")
ax_main.grid(True)
ax_main.set_xlabel("Subject Index (Sorted by Median)", fontsize=16)
ax_main.set_ylabel(f"Bilirubin ({bili_unit})", fontsize=16)
ax_main.set_title("Bilirubin Distributions: Within vs Across Subjects", fontsize=16)
ax_main.legend(fontsize=16)

# Y-limits (match ADaM feel but don’t clip real data)
ax_main.set_ylim([-1,40])

# --- Histogram (right) ---
ax_hist = plt.subplot(gs[1], sharey=ax_main)
ax_hist.hist(y_plot, bins=bins, orientation="horizontal", alpha=0.7)

if bars_right:
    ax_hist.yaxis.tick_right()
    ax_hist.yaxis.set_label_position("right")
else:
    ax_hist.invert_xaxis()
    ax_hist.yaxis.tick_left()
    ax_hist.yaxis.set_label_position("left")

ax_hist.set_xlabel("Count")
ax_hist.set_ylabel(f"Bilirubin ({bili_unit})", fontsize=14, labelpad=10)
ax_hist.tick_params(axis="y", labelsize=12)
ax_hist.set_title("Overall Distribution", fontsize=14)
ax_hist.grid(True)

# --- Save ---

save_index += 1
fig_str = f"{save_index:02d}"
out_path = FIG_DIR / f"{fig_str}_bilirubin_quantiles_plus_hist.jpg"
savefig(out_path, dpi=100)
print(f"Saved: {out_path}")

Note that one patient had values over 120 umol/L, which are clipped out from the Y-axis.

<img src="../figures/13_bilirubin_quantiles_plus_hist.jpg?v=1" alt="Available Lab Params" width="1000">
